# SQL Server Ground to Cloud Security Workshop

_Last Updated: 25 July 2022_

In [this workshop](https://github.com/David-Seis/SecureYourAzureData) you'll cover the basics of securing SQL Server installations and databases, from on-premises systems to Microsoft Azure deployments.

_Note that this Notebook is an example only; and should not be used in production. It should be evaluated as a training implement only, and you should use these scripts and statements in a read-only fashion on a test system not connected to your production environment._